In [6]:
import os
os.environ['GROQ_API_KEY'] = 'ADD YOUR API KEY HERE'

In [7]:
#!pip install groq
#pip install agentic_patterns
#pip install httpx==0.27.0

import json
import requests
from agentic_patterns.tool_pattern.tool import tool
from agentic_patterns.tool_pattern.tool_agent import ToolAgent

In [8]:
# Function to fetch top stories from Hacker News.

def fetch_top_hacker_news_stories(top_n: int):
    """
    Fetch the top stories from Hacker News.

    This function retrieves the top `top_n` stories from Hacker News using the Hacker News API.
    Each story contains the title, URL, score, author, and time of submission. The data is fetched
    from the official Firebase Hacker News API, which returns story details in JSON format.

    Args:
        top_n (int): The number of top stories to retrieve.
    """
    top_stories_url = 'https://hacker-news.firebaseio.com/v0/topstories.json'

    try:
        response = requests.get(top_stories_url)
        response.raise_for_status()  # Check for HTTP errors.

        # Get the top story IDs.
        top_story_ids = response.json()[:top_n]

        top_stories = []

        # For each story ID, fetch the story details.
        for story_id in top_story_ids:
            story_url = f'https://hacker-news.firebaseio.com/v0/item/{story_id}.json'
            story_response = requests.get(story_url)
            story_response.raise_for_status()  # Check for HTTP errors.
            story_data = story_response.json()

            # Append the story title and URL (or other relevant info) to the list.
            top_stories.append({
                'title': story_data.get('title', 'No title'),
                'url': story_data.get('url', 'No URL available'),
            })

        return json.dumps(top_stories)

    except requests.exceptions.RequestException as e:
        print(f"An error occurred: {e}")
        return []

In [9]:
json.loads(fetch_top_hacker_news_stories(top_n=3)) # Fetch the top 3 stories and parse the JSON string into a Python object.
hn_tool = tool(fetch_top_hacker_news_stories) # Wraps the function in a tool decorator from the agentic_patterns.tool_pattern library.
hn_tool.name # Return the tool name, the function name.
json.loads(hn_tool.fn_signature) # Load the function's argument schema {"top_n": int}.

{'name': 'fetch_top_hacker_news_stories',
 'description': '\n    Fetch the top stories from Hacker News.\n\n    This function retrieves the top `top_n` stories from Hacker News using the Hacker News API. \n    Each story contains the title, URL, score, author, and time of submission. The data is fetched \n    from the official Firebase Hacker News API, which returns story details in JSON format.\n\n    Args:\n        top_n (int): The number of top stories to retrieve.\n    ',
 'parameters': {'properties': {'top_n': {'type': 'int'}}}}

In [10]:
tool_agent = ToolAgent(tools=[hn_tool], model="llama3-8b-8192") # Create a ToolAgent to run the tool/function based on the user message.

# The agent parses the user message, detects the intent (call fetch_top_hacker_news_stories with top_n=3), and returns the output.
output = tool_agent.run(user_msg="Please share the top 3 Hacker News stories.")
print(output)


Using Tool: fetch_top_hacker_news_stories

Tool call dict: 
{'name': 'fetch_top_hacker_news_stories', 'arguments': {'top_n': 3}, 'id': '1'}

Tool result: 
[{"title": "Introducing tmux-rs", "url": "https://richardscollin.github.io/tmux-rs/"}, {"title": "Launch HN: K-Scale Labs (YC W24) \u2013 Open-Source Humanoid Robots", "url": "No URL available"}, {"title": "Poor Man's Back End-as-a-Service (BaaS), Similar to Firebase/Supabase/Pocketbase", "url": "https://github.com/zserge/pennybase"}]
A Hacker News style story!

According to the observation, the top 3 Hacker News stories are:

1. **Introducing tmux-rs**: A new Rust library for managing tmux sessions. [https://richardscollin.github.io/tmux-rs/](https://richardscollin.github.io/tmux-rs/)
2. **Launch HN: K-Scale Labs (YC W24) – Open-Source Humanoid Robots**: A new open-source humanoid robot project from Y Combinator W24 batch. (No URL available)
3. **Poor Man's Back End-as-a-Service (BaaS), Similar to Firebase/Supabase/Pocketbase**: An